In [1]:
import tensorflow as tf
import numpy as np
import os
import scipy.io as spio
from matplotlib import pyplot as plt
from imageio import imread

In [2]:
# define base paths for pascal the original VOC dataset training images
base_dataset_dir_voc = '/Users/udaysawhney/Desktop/create_dataset/'
images_folder_name_voc = "dataset/"
annotations_folder_name_voc = "SegmentationClass/"
images_dir_voc = os.path.join(base_dataset_dir_voc, images_folder_name_voc)
print(images_dir_voc)
annotations_dir_voc = os.path.join(base_dataset_dir_voc, annotations_folder_name_voc)

/Users/udaysawhney/Desktop/create_dataset/dataset/


In [3]:
def get_files_list(base_dataset_dir, images_folder_name, annotations_folder_name, filename):
    images_dir = os.path.join(base_dataset_dir, images_folder_name)
    annotations_dir = os.path.join(base_dataset_dir, annotations_folder_name)

    file = open(filename, 'r')
    images_filename_list = [line for line in file]
    return images_filename_list

In [4]:
test_images_filename_list = get_files_list(base_dataset_dir_voc, images_folder_name_voc, annotations_folder_name_voc, "/Users/udaysawhney/Desktop/create_dataset/custom_test.txt")
print("Total number of training images:", len(test_images_filename_list))

Total number of training images: 32


In [5]:
TRAIN_DATASET_DIR="./tfrecords/"
if not os.path.exists(TRAIN_DATASET_DIR):
    os.mkdir(TRAIN_DATASET_DIR)
    
TEST_FILE = 'test123.tfrecords'
test_writer = tf.python_io.TFRecordWriter(os.path.join(TRAIN_DATASET_DIR,TEST_FILE))

In [6]:
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [7]:
def read_annotation_from_xml_file(annotations_dir, image_name):
    annotations_path = os.path.join(annotations_dir, (image_name.strip() + ".xml"))
    print(annotations_path)
    xml = spio.loadmat(annotations_path)
    
    print('HERE NIGA')
    img = xml['GTcls']['Segmentation'][0][0]
    return img

In [8]:
for i, image_name in enumerate(test_images_filename_list):
    image_np = imread(os.path.join(images_dir_voc, image_name.strip() + ".jpg"))

In [9]:
def create_tfrecord_dataset(filename_list, writer):

    # create training tfrecord
    read_imgs_counter = 0
    for i, image_name in enumerate(filename_list):
        image_name = str(image_name)

        try:
            image_np = imread(os.path.join(images_dir_voc, image_name.strip() + ".jpg"))
        except FileNotFoundError:
            try:
                # read from Pascal VOC path
                image_np = imread(os.path.join(images_dir_voc, image_name.strip() + ".jpg"))
            except FileNotFoundError:
                print(os.path.join(images_dir_voc, image_name.strip() + ".jpg"))
                print("File:",image_name.strip(),"not found.")
                continue
        try:
            annotation_np = read_annotation_from_xml_file(annotations_dir_voc, image_name)
        except FileNotFoundError:
            # read from Pascal VOC path
            try:
                annotation_np = imread(os.path.join(annotations_dir_voc, image_name.strip() + ".png"))
                print('SUCCESS!!')
            except FileNotFoundError:
                print("File:",image_name.strip(),"not found lolololol.")
                continue
            
        read_imgs_counter += 1
        image_h = image_np.shape[0]
        image_w = image_np.shape[1]

        img_raw = image_np.tostring()

        example = tf.train.Example(features=tf.train.Features(feature={
                'height': _int64_feature(image_h),
                'width': _int64_feature(image_w),
                'image_raw': _bytes_feature(img_raw),
        }))

        writer.write(example.SerializeToString())
    
    print("End of TfRecord. Total of image written:", read_imgs_counter)
    writer.close()

In [10]:
create_tfrecord_dataset(test_images_filename_list, test_writer)

/Users/udaysawhney/Desktop/create_dataset/SegmentationClass/0000_nir.xml
SUCCESS!!
/Users/udaysawhney/Desktop/create_dataset/SegmentationClass/0005_nir.xml
SUCCESS!!
/Users/udaysawhney/Desktop/create_dataset/SegmentationClass/0006_nir.xml
SUCCESS!!
/Users/udaysawhney/Desktop/create_dataset/SegmentationClass/0007_nir.xml
SUCCESS!!
/Users/udaysawhney/Desktop/create_dataset/SegmentationClass/0008_nir.xml
SUCCESS!!
/Users/udaysawhney/Desktop/create_dataset/SegmentationClass/0017_nir.xml
SUCCESS!!
/Users/udaysawhney/Desktop/create_dataset/SegmentationClass/0020_nir.xml
SUCCESS!!
/Users/udaysawhney/Desktop/create_dataset/SegmentationClass/0025_nir.xml
SUCCESS!!
/Users/udaysawhney/Desktop/create_dataset/SegmentationClass/0026_nir.xml
SUCCESS!!
/Users/udaysawhney/Desktop/create_dataset/SegmentationClass/0027_nir.xml
SUCCESS!!
/Users/udaysawhney/Desktop/create_dataset/SegmentationClass/0033_nir.xml
SUCCESS!!
/Users/udaysawhney/Desktop/create_dataset/SegmentationClass/0035_nir.xml
SUCCESS!!
/Use